In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Houston/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/16203-beechnut-st-houston-tx-77083-27580627',
 'https://www.trulia.com/home/6811-alum-rock-ln-houston-tx-77048-161439693',
 'https://www.trulia.com/home/14723-oak-bend-dr-houston-tx-77079-28123622',
 'https://www.trulia.com/home/5305-weslayan-st-houston-tx-77005-123224631',
 'https://www.trulia.com/home/3538-noah-st-houston-tx-77021-305346158',
 'https://www.trulia.com/home/9911-cedarhurst-dr-houston-tx-77096-28066590',
 'https://www.trulia.com/home/5022-natural-bridge-dr-kingwood-tx-77345-28506973',
 'https://www.trulia.com/home/16207-sommerall-creek-ln-houston-tx-77084-245772575',
 'https://www.trulia.com/home/1033-herkimer-st-houston-tx-77008-27761522',
 'https://www.trulia.com/home/11414-yorkshire-oaks-dr-houston-tx-77065-28461081',
 'https://www.trulia.com/home/3022-cherry-mill-ct-houston-tx-77059-28525301',
 'https://www.trulia.com/home/4229-bettis-dr-houston-tx-77027-55474743',
 'https://www.trulia.com/home/1615-milford-st-houston-tx-77006-27828689'

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/16203-beechnut-st-houston-tx-77083-27580627


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [16]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['16203 Beechnut St',
  '6811 Alum Rock Ln',
  '14723 Oak Bend Dr',
  '5305 Weslayan St',
  '3538 Noah St',
  '9911 Cedarhurst Dr',
  '5022 Natural Bridge Dr',
  '16207 Sommerall Creek Ln',
  '1033 Herkimer St',
  '11414 Yorkshire Oaks Dr',
  '3022 Cherry Mill Ct',
  '4229 Bettis Dr',
  '1615 Milford St',
  '1402 Lakin St #1 & 2',
  '1126 Pear Tree Ln',
  '6003 Drenner Park Ln',
  '8329 Charlesmont St #343',
  '6302 Briar Glade Dr',
  '12012 Royal Oaks Run Dr',
  '540 Rivergrove Dr',
  '6905 Wheatley St',
  '1711 Old Spanish Trl #339',
  '611 E  7th 1/2 St',
  '6344 Brompton Rd',
  '16002 Buccaneer Ln',
  '9103 Braeburn Valley Dr',
  '15639 Weldon Dr',
  '5226 Calle Cordoba Pl',
  'Spectra Plan in Imperial Forest by Starlight',
  '1411 Spillers Ln',
  '802 Walnut Bend Ln',
  '8127 Canyon St',
  '5206 Trail Lake Dr',
  'Green River Dr',
  '1701 Hermann Dr #21A',
  '15706 Dawnbrook Dr',
  '1012 Peden St #B',
  '6114 Rollingbrook Dr',
  '210 Woodcombe Dr',
  '16314 Pelican Bea

In [17]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,16203 Beechnut St,1 Bath,874 sqft,1982,Garage,"$194,000"
1,6811 Alum Rock Ln,2 Baths,"1,423 sqft",2016,2 Car Garage,"$231,950"
2,14723 Oak Bend Dr,3 Baths,"2,842 sqft",1967,Garage,"$974,000"
3,5305 Weslayan St,5 Baths,"2,867 sqft",2014,2 Car Garage,"$789,000"
4,3538 Noah St,4 Baths,"1,827 sqft",2020,2 Car Garage,"$329,000"
5,9911 Cedarhurst Dr,3 Baths,"2,624 sqft",1960,Garage,"$719,990"
6,5022 Natural Bridge Dr,3 Baths,"2,542 sqft",1991,Garage,"$369,999"
7,16207 Sommerall Creek Ln,3 Baths,"2,164 sqft",2017,2 Car Garage,"$375,000"
8,1033 Herkimer St,3 Baths,"2,520 sqft",2008,2 Car Garage,"$759,000"
9,11414 Yorkshire Oaks Dr,2 Baths,"1,875 sqft",1995,2 Car Garage,"$295,000"


In [18]:
# Add location to dataframe
df['Location'] = 'Houston'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,16203 Beechnut St,1 Bath,874 sqft,1982,Garage,"$194,000",Houston
1,6811 Alum Rock Ln,2 Baths,"1,423 sqft",2016,2 Car Garage,"$231,950",Houston
2,14723 Oak Bend Dr,3 Baths,"2,842 sqft",1967,Garage,"$974,000",Houston
3,5305 Weslayan St,5 Baths,"2,867 sqft",2014,2 Car Garage,"$789,000",Houston
4,3538 Noah St,4 Baths,"1,827 sqft",2020,2 Car Garage,"$329,000",Houston
5,9911 Cedarhurst Dr,3 Baths,"2,624 sqft",1960,Garage,"$719,990",Houston
6,5022 Natural Bridge Dr,3 Baths,"2,542 sqft",1991,Garage,"$369,999",Houston
7,16207 Sommerall Creek Ln,3 Baths,"2,164 sqft",2017,2 Car Garage,"$375,000",Houston
8,1033 Herkimer St,3 Baths,"2,520 sqft",2008,2 Car Garage,"$759,000",Houston
9,11414 Yorkshire Oaks Dr,2 Baths,"1,875 sqft",1995,2 Car Garage,"$295,000",Houston


In [19]:
df.to_excel('dataframe_houston.xlsx', index=False)